# Atualizar
- Melhorar prompt 
- Verificar Persistência dos dados 
- Testar diferentes modelos
- Alterar o parser de pdf para o parser do llama_index


In [1]:
import gradio as gr
import chromadb
from glob import glob
from PyPDF2 import PdfReader
from os import path, getenv
from dotenv import load_dotenv
from llama_index.core.schema import TextNode
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.core import VectorStoreIndex, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore

## Classe Config
- Define as variáveis de ambiente necessárias para a execução do programa de um arquivo .env.
- Iniciliza a conexão com o ChromaDB.

In [3]:
class Config:
    def __init__(self):
        
        load_dotenv()
        self.llm_model = getenv('llm_model')
        self.embedding_model = getenv('embedding_model')
        self.api_key = getenv('api_key')
        self.db_path = getenv('db_path')
        self.input_dir = getenv('input_dir')
        self.collection_name = getenv('collection_name')
        self.connection = None

    def connect_db(self):
        try:
            db_client = chromadb.PersistentClient(path=self.db_path)
            collection = db_client.get_or_create_collection(self.collection_name)
            self.connection = ChromaVectorStore(chroma_collection=collection)
            
            print("Conexão ao ChromaDB estabelecida com sucesso!")
            return self.connection
        except Exception as e:
            raise Exception(f"Erro ao conectar ao banco de dados: {str(e)}")
        




Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/3 [00:00<?, ?it/s]

## Classe GenerateIndex
- Faz o pré processamento dos pdfs dividindo o texto em chunks de 100 palavras
- Gera os embbedings e armazena em um banco de dados chromadb

In [ ]:
class GenarateIndex:
    def __init__(self):
        self.config = Config()
        self.vector_store = self.config.connect_db()
        self.llm_model = self.config.llm_model
        self.embed_model_name = self.config.embedding_model
        self.input_dir = self.config.input_dir
        self.api_key = self.config.api_key
        self.nodes = None

    def generate_index(self):
        try:
            self.embedding_model = HuggingFaceEmbedding(model_name=self.embed_model_name, embed_batch_size = 20)
            self.index = VectorStoreIndex(
                nodes = self.nodes,
                vector_store=self.vector_store,
                embed_model=self.embedding_model,
                show_progress=True,

            )
            print("Índice gerado com sucesso!")
            return self.index
        except Exception as e:
            raise Exception(f"Erro ao gerar índice: {str(e)}")

    def split_into_chunks(self, text, chunk_size=100, overlap=10):
        """
        Divide o texto em chunks de palavras com overlap especificado.

        :param text: Texto completo que será dividido.
        :param chunk_size: Número de palavras em cada chunk.
        :param overlap: Quantidade de palavras que se sobrepõem entre chunks.
        :return: Lista de chunks de texto.
        """
        words = text.split()  # Divide o texto em uma lista de palavras
        chunks = []
        start = 0
        while start < len(words):
            end = start + chunk_size
            chunk = " ".join(words[start:end])  # Junta as palavras para formar o chunk
            chunks.append(chunk)
            start += chunk_size - overlap  # Avança, considerando o overlap
        return chunks
    
    def process_pdfs(self, input_dir):
        files = glob(f"{input_dir}/*.pdf")
        text_nodes = []
        try:
            for file in files:
                reader = PdfReader(file)
                text = ""
                
                for page in reader.pages:
                    extracted_text = page.extract_text().replace('\n', ' ').replace('  ', ' ')
                    print
                    if extracted_text:
                        text += extracted_text + " "
                
                if not text.strip():
                    raise ValueError(f"O PDF {file} parece não conter texto processável.")
                
                chunks = self.split_into_chunks(text)
                
                # Armazenando os chunks de texto e metadados
                for chunk in chunks:
                    cont =+ 1
                    text_nodes.append(TextNode(
                    text=chunk,
                    metadata={"filename": file, "num_chunk": cont},
                    
                ))
                # Armazenando o texto extraído e metadados
                
                print(f"Texto extraído do arquivo {file} com sucesso!")
            self.nodes = list(text_nodes)
            return self.nodes
        except Exception as e:
            raise Exception(f"Erro ao transformar texto: {str(e)}")
        
    def execute(self):
        try:
            self.process_pdfs(self.input_dir)
            self.generate_index()
            return self.index
        except Exception as e:
            raise Exception(f"Erro ao executar o processo: {str(e)}")
        


## Classe Chatbot
- Contém as configurações do modelo de llm
- Realiza a busca por similaridade usando o método as_query_engine() do llama_index
- Envia a pergunta + a resposta da query para o llm gerar a resposta contextualizada

In [4]:
class ChatBot:
    def __init__(self, index):
        self.config = Config()
        self.api_key = self.config.api_key
        self.llm_model = self.config.llm_model
        self.index = index
        

    def initialize_llm(self):
        try:
            self.prompt = (
                    "Essas são suas diretrizes para interagir com os usuários: "
                    "Seu nome é BOOT. "
                    "Responda com base no que sabe e adapte suas respostas de forma a se encaixar perfeitamente na pergunta ou necessidade do usuário. "
                    "Não use os termos 'Resposta:' ou 'Pergunta:' ao iniciar suas mensagens. "
                    "Você é um assistente virtual projetado para fornecer informações sobre diversos tópicos. "
                    "Responda exclusivamente em português, garantindo clareza e precisão. "
                    "Ofereça respostas educadas, amigáveis e sempre no mesmo tom e idioma utilizado pelo usuário. "
                    "Caso o usuário inicie a conversa com uma saudação ou pergunte quem você é, apresente-se de forma cordial e simpática. "
                    "Seja educado em todas as interações, objetivo e direto "
                    "Sempre atenda aos pedidos do usuário em português."
                    )
            Settings.llm = HuggingFaceInferenceAPI(model_name=self.llm_model, 
                                                   token=self.api_key, 
                                                   system_prompt=self.prompt,
                                                   generate_kwargs={"temperature": 0.7})

            chat_engine = self.index.as_chat_engine(
                chat_mode="condense_question",
                verbose=False
            )

            return chat_engine
            
        except Exception as e:
            raise Exception(f"Erro ao inicializar Modelo: {str(e)}")
        
    def search(self, query):
        try:
            query_engine = self.index.as_query_engine(
                llm=HuggingFaceInferenceAPI(model_name=self.llm_model, token=self.api_key),
                response_mode="tree_summarize",
                max_results=5,
                verbose=False
            )
            response = query_engine.query(query)
            context = " || ".join([doc.node.text for doc in response.source_nodes])
            return response, context
        except Exception as e:
            raise Exception(f"Erro ao realizar a busca: {str(e)}")
        
        
    def respond(self, message, history=None):
        """Gera uma resposta baseada no contexto da pesquisa."""
        try:
            
            # Busca o contexto no ChromaDB
            chat_engine = self.initialize_llm()
            response, context = self.search(message)

            message_context = f"Contexto: {context} || Pergunta: {message}"

            # Gera a resposta usando o motor de chat
            response = chat_engine.chat(message_context)

            return response.response
        
        except Exception as e:
            return f"Erro ao processar mensagem: {str(e)}"

    def interface(self): 
        demo = gr.ChatInterface(
            fn=self.respond,
            type="messages", 
            examples=["hello", "hola", "merhaba", "oi"], 
            title="Boot",
        )
        return demo.launch()

### Se conecta ao db e armazena o index (embbeding)

In [ ]:
generate_index = GenarateIndex()
index = generate_index.execute()

### Utliza o index para realizar as interações

In [ ]:
chatbot = ChatBot(index)
chatbot.interface()